<a href="https://colab.research.google.com/github/jaycrossler/ai-training/blob/main/CrewAI_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install langchain_community colab-xterm crewai 'crewai[tools]'
%load_ext colabxterm
# If asked to restart, accept and run this cell again

The colabxterm extension is already loaded. To reload it, use:
  %reload_ext colabxterm


In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [13]:
import subprocess
import time

In [30]:
server = subprocess.Popen(['ollama', 'serve'])
#time.sleep(60) # To make sure ollama is ready in subsequent cell if you are running all not cell at a time


In [7]:
llama3 = subprocess.Popen(['ollama', 'run', 'llama3'])

In [31]:
!ollama list

NAME             ID              SIZE      MODIFIED       
llama3:latest    365c0bd3c000    4.7 GB    56 minutes ago    


In [32]:
from langchain_community.llms import Ollama

# Initialize an instance of the Ollama model
llm = Ollama(model="llama3")

# Invoke the model to generate responses
response = llm.invoke("Tell me a joke")
print(response)

Here's one:

Why couldn't the bicycle stand up by itself?

(wait for it...)

Because it was two-tired!

Hope that made you laugh!


In [33]:
from crewai import Agent, Task, Crew, Process, LLM
from langchain_community.llms import Ollama
#llm = Ollama(model="llama3")

llm_o=LLM(model="ollama/llama3", base_url="http://localhost:11434")

researcher = Agent(
        role = "Researcher",
        goal = f"""To collaborate with your colleages to gather their views
        and opitions and get a final answer to complex questions""",
        verbose = True, # This way we can see the agents thoughts and messages
        #human_input=True, # Enable this if you want to be able to ask the user questions
        allow_delegation=True, # This allows to delegate tasks to other agents
        llm=llm_o,
        backstory = """You are an experienced researcher.
        You collaborate with expert colleages to get their views.
        You synthesize information in a simple and understandable way.
        """
    )


architecht = Agent(
        role = "System Architecht",
        goal = f"To collaborate with your colleages and provide your views as a system architecht",
        verbose = True, # This way we can see the agents thoughts and messages
        allow_delegation=False,
        llm=llm_o,
        backstory = """You are an experienced government software and system architecht.
        You can find answers to complex questions and work with colleagues.
        """
    )

commander = Agent(
        role = "commander",
        goal = f"To collaborate with your colleages and provide your views as a military leader and commander",
        verbose = True,# This way we can see the agents thoughts and messages
        allow_delegation=False,
        llm=llm_o,
        backstory = """You are an experienced Military Commander.
        You can find answers to complex questions and work with colleagues.
        """
    )

economist = Agent(
        role = "Economist",
        goal = f"To collaborate with your colleages and provide your views as an economist and acquisition expert",
        verbose = True,# This way we can see the agents thoughts and messages
        allow_delegation=False,
        llm=llm_o,
        backstory = """You are an experienced economis and government program acquisition expert.
        You can find answers to complex questions and work with colleagues.
        """
    )

LLM value is already an LLM object
LLM value is already an LLM object
LLM value is already an LLM object
LLM value is already an LLM object


In [34]:
task= Task(
    description="Describe a system architecture for a Government drone project that a Commander, Economist, and System Architecht can all agree upon?",
    expected_output="""
    The answer to the question: "Describe a system architecture for a Government drone project that a Commander, Economist, and System Architecht can all agree upon?"
    """,
    max_inter=5, #This defines the max amount of iterations an agent can perform to avoid infinite loops. Max=25
    #human_input=True,
    agent=researcher # We will delegate the task initially to the researcher
    )

In [35]:
crew = Crew(
    agents=[researcher,architecht,commander,economist],
    model="ollama/llama3",
    tasks=[task],
    verbose=True, # This way we can see the agents thoughts and messages
    process=Process.sequential)

result = crew.kickoff()

# Agent: Researcher
## Task: Describe a system architecture for a Government drone project that a Commander, Economist, and System Architecht can all agree upon?
 

I encountered an error while trying to use the tool. This was the error: unhashable type: 'dict'.
 Tool Delegate work to coworker accepts these inputs: Tool Name: Delegate work to coworker
Tool Arguments: {'task': {'description': 'The task to delegate', 'type': 'str'}, 'context': {'description': 'The context for the task', 'type': 'str'}, 'coworker': {'description': 'The role/name of the coworker to delegate to', 'type': 'str'}}
Tool Description: Delegate a specific task to one of the following coworkers: System Architecht, commander, Economist
The input to this tool should be the coworker, the task you want them to do, and ALL necessary context to execute the task, they know nothing about the task, so share absolute everything you know, don't reference things but instead explain them.



# Agent: Researcher
## Using tool: 